<a href="https://colab.research.google.com/github/Shk-tech/my-colab-learning-material/blob/main/2507_korea_pytorch_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yasserh/titanic-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/titanic-dataset


In [ ]:
!cp -r /kaggle/input/titanic-dataset .

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('titanic-dataset/Titanic-Dataset.csv')
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
df['Pclass'].unique()

array([3, 1, 2])

In [ ]:
len(df[''].unique())

7

In [ ]:
df['Fare'].max()

512.3292

In [ ]:
enable_x = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked']
enable_y = ['Survived']
df_x = df[enable_x]
df_y = df[enable_y]

In [ ]:
# 전처리... float -> 결측치제거, one-hot, scaling...
# 결측치 제거
# 수치형 데이터 - mean, median
# 카테고리형 데이터 - Embarked ...??? 최빈값.. 채운다?? Unknown

In [ ]:
df_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       714 non-null    float64
 3   Fare      891 non-null    float64
 4   Embarked  889 non-null    object 
dtypes: float64(2), int64(1), object(2)
memory usage: 34.9+ KB


In [ ]:
num_attr = ['Pclass', 'Age', 'Fare']
cat1_attr = ['Sex']
cat2_attr = ['Embarked']

In [ ]:
df_x[num_attr]
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

#Sequential
num_p = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("standardize", StandardScaler()),
])

num_p.fit(df_x[num_attr])

Pipeline(steps=[('impute', SimpleImputer(strategy='median')),
                ('standardize', StandardScaler())])

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat2_p = Pipeline([
    ('unknown imputer',SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('enc', OneHotEncoder(sparse_output=False))
])

cat2_p.fit(df_x[cat2_attr])

Pipeline(steps=[('unknown imputer',
                 SimpleImputer(fill_value='Unknown', strategy='constant')),
                ('enc', OneHotEncoder(sparse_output=False))])

In [ ]:
from sklearn.compose import ColumnTransformer
preprocessing = ColumnTransformer([
    ("num", num_p, num_attr),
    ("cat1", OneHotEncoder(sparse_output=False), cat1_attr),
    ("cat2", cat2_p, cat2_attr)
])

preprocessing.fit(df_x)
preprocessing.transform(df_x).shape

(891, 9)

# 딥러닝

In [ ]:
# train - valid....
# preproc -> train-valid split
# train-valid split -> preproc
ri = np.random.permutation(len(df_x))
test_size = int(len(df_x) * 0.1)
x_valid = df_x.iloc[ri[:test_size]]
x_train = df_x.iloc[ri[test_size:]]
y_valid = df_y.iloc[ri[:test_size]]
y_train = df_y.iloc[ri[test_size:]]

x_valid = preprocessing.transform(x_valid).astype(np.float32) # numpy변환까지 해주는데...
x_train = preprocessing.transform(x_train).astype(np.float32)
y_valid = y_valid.values.astype(np.float32) #numpy or tensor로 변환해줘야 한다. shapeN , 1
y_train = y_train.values.astype(np.float32) #multi-classification -> float(x) int

In [ ]:
# 1. regression     2. bin-classification   3. multi-classification
# 1. regression
#  - y값에 대해서도 scaling 적용
#  - float, (N, 1)

# 2. bin-classification
#  - 0 or 1 로 만든다.
#  - sklearn(int), pytorch(float 0.0/1.0)
#  - shape (N, 1)

# 3. multi-classification
#  - int 0~C-1 (onehot encoding 하지 않음!)
#  - shape (N,)

In [ ]:
loader_train = torch.utils.data.DataLoader(list(zip(x_train, y_train)), batch_size=16, shuffle=True, drop_last=True)
loader_valid = torch.utils.data.DataLoader(list(zip(x_train, y_train)), batch_size=16)

for x, y in loader_train:
    break
x.shape, y.shape, x.dtype, y.dtype

(torch.Size([16, 9]), torch.Size([16, 1]), torch.float32, torch.float32)

In [ ]:
# model = nn.Sequential(
#     nn.Linear(9, 128),
#     nn.Linear(128, 128),
#     nn.Linear(128, 1),
# )

model = nn.Sequential(
    nn.LazyLinear(128),
    nn.LazyBatchNorm1d(),
    nn.SiLU(),
    nn.LazyLinear(128),
    nn.LazyBatchNorm1d(),
    nn.SiLU(),
    nn.LazyLinear(1),
    nn.Sigmoid() # 이진 분류에서 마지막에 넣어줌.
)

opt = torch.optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()

#lazy를 쓸 때는 테스트 필수
prob = model(x)
loss = loss_fn(prob, y)
loss.backward()

In [ ]:
for epoch in range(30):
    model.train()
    t_acc = []
    for x, y in loader_train:
        #1. feed-forward(pred)
        #2. loss(pred, y)
        #3. backprop(grad)
        #4. update
        pred = model(x)
        loss = loss_fn(pred, y)
        opt.zero_grad()
        loss.backward()
        opt.step()
        #metrics - 현재 학습 상황...
        acc = ((pred > 0.5).float() == y).float().mean()
        t_acc.append(acc.item())
        m_acc = np.mean(t_acc)
        print('\r e=%d l=%f' % (epoch, m_acc), end='') #???
    print()
    model.eval()
    t_acc = []
    for x, y in loader_valid:
        #1. feed-forward(pred)
        #2. loss(pred, y)
        #3. backprop(grad)
        #4. update
        with torch.no_grad():
            pred = model(x)
        loss = loss_fn(pred, y)
        # opt.zero_grad()
        # loss.backward()
        # opt.step()
        #metrics - 현재 학습 상황...
        acc = ((pred > 0.5).float() == y).float().mean()
        t_acc.append(acc.item())
        m_acc = np.mean(t_acc)
        print('\r                   e=%d l=%f' % (epoch, m_acc), end='') #???
    print()

 e=0 l=0.832500
                   e=0 l=0.828431
 e=1 l=0.841250
                   e=1 l=0.823529
 e=2 l=0.847500
                   e=2 l=0.832108
 e=3 l=0.838750
                   e=3 l=0.829657
 e=4 l=0.843750
                   e=4 l=0.827206
 e=5 l=0.826250
                   e=5 l=0.828431
 e=6 l=0.830000
                   e=6 l=0.830882
 e=7 l=0.846250
                   e=7 l=0.828431
 e=8 l=0.842500
                   e=8 l=0.840686
 e=9 l=0.830000
                   e=9 l=0.828431
 e=10 l=0.837500
                   e=10 l=0.828431
 e=11 l=0.840000
                   e=11 l=0.828431
 e=12 l=0.840000
                   e=12 l=0.832108
 e=13 l=0.832500
                   e=13 l=0.832108
 e=14 l=0.825000
                   e=14 l=0.821078
 e=15 l=0.842500
                   e=15 l=0.823529
 e=16 l=0.831250
                   e=16 l=0.829657
 e=17 l=0.837500
                   e=17 l=0.825980
 e=18 l=0.832500
                   e=18 l=0.830882
 e=19 l=0.842500
               

# 테스트

In [ ]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 65.6 MB/s eta 0:00:00


In [ ]:
sample_input = torch.rand((1, 9)).float() # 입력 타입과 shape을 지정

torch.onnx.export(
    model,                  # PyTorch Model
    sample_input,            # Input tensor, dtype=uint8, shape=(1, 1, 3)
    "titanic.onnx",        # Output file (eg. 'output_model.onnx')
    opset_version=17,       # Operator support version
    input_names=['input'],   # Input tensor name (arbitary)
    output_names=['output'], # Output tensor name (arbitary)
    dynamic_axes={'input':[0]}
)

/usr/local/lib/python3.11/dist-packages/torch/onnx/utils.py:1828: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input input
  warnings.warn(


In [ ]:
import pickle
pickle.dump(preprocessing, open('titanic_prep.pickle', 'wb'))

In [ ]:
prep = pickle.load(open('titanic_prep.pickle', 'rb'))

In [ ]:
prep.transform(df_x)

array([[ 0.82737724, -0.56573646, -0.50244517, ...,  0.        ,
         1.        ,  0.        ],
       [-1.56610693,  0.66386103,  0.78684529, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.82737724, -0.25833709, -0.48885426, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 0.82737724, -0.1046374 , -0.17626324, ...,  0.        ,
         1.        ,  0.        ],
       [-1.56610693, -0.25833709, -0.04438104, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.82737724,  0.20276197, -0.49237783, ...,  1.        ,
         0.        ,  0.        ]])

In [ ]:
df_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       714 non-null    float64
 3   Fare      891 non-null    float64
 4   Embarked  889 non-null    object 
dtypes: float64(2), int64(1), object(2)
memory usage: 34.9+ KB


In [ ]:
import sklearn
sklearn.__version__

'1.6.1'

In [ ]:
df_x['Embarked'].unique()

array(['S', 'C', 'Q', nan], dtype=object)